In [1]:
import numpy as np
import pandas as pd

data_x_df = pd.read_csv('data_X.csv')
data_t_df = pd.read_csv('data_T.csv')

In [2]:
input_size = data_x_df.shape[0]
try:
    if data_x_df.shape[0] == data_t_df.shape[0]:
        pass
except:
    print('Numbers of data size mismatch!!')

# Turn dataset into numpy form and random shuffle
data_df = pd.merge(data_x_df, data_t_df, left_index=True, right_index=True)
data = np.array(data_df)
np.random.seed(1)
np.random.shuffle(data)

# Divide the dataset into training and validation
train_x = data[:int(0.9 * input_size), 0:8]
train_t = data[:int(0.9 * input_size), 8]

valid_x = data[int(0.9 * input_size):, 0:8]
valid_t = data[int(0.9 * input_size):, 8]

In [3]:
def root_mean_square_error(x, w, t):
    y_xw = np.dot(x, w)
    error = 0.5 * np.mean((y_xw - t) ** 2)
    rms_error = np.sqrt(2 * np.array(error))
    return rms_error

# w_ls = (x_T * x)^(-1) * x_T * y
def least_square(x, t):
    tmp_inv = np.linalg.pinv(np.dot(x.T, x))
    w_ls = np.dot(np.dot(tmp_inv, x.T), t)
    return w_ls

# w_rls = (x_T * x + lambda * I)^(-1) * x_T * y
def regularize_least_square(x, t, lambda_rls=1):
    tmp = np.dot(x.T, x)
    tmp_inv = np.linalg.pinv(tmp + lambda_rls * np.eye(tmp.shape[0]))
    w_rls = np.dot(np.dot(tmp_inv, x.T), t)
    return w_rls

def basis_function(x, order=1, features=8):
    phi_x = np.full((x.shape[0], 1), 1)
    
    for ord in range(1, order + 1):
        #for comb in list(repeat_combination(range(features), ord)):
        for comb in list(product_permute(range(features), repeat=ord)):
            product = np.prod(x[:, comb], axis=1)
            new_col = np.expand_dims(product, axis=1)
            phi_x = np.concatenate((phi_x, new_col), axis=1)
    return phi_x

def poly_basis_function(x, order=1, features=8):
    phi_x = np.full((x.shape[0], 1), 1)
    
    for ord in range(1, order + 1):
        for deg in range(features):
            product = np.prod(x[:, [deg] * ord], axis=1)
            new_col = np.expand_dims(product, axis=1)
            phi_x = np.concatenate((phi_x, new_col), axis=1)
    return phi_x

def repeat_combination(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    if not n and r:
        return
    indices = [0] * r
    yield tuple(pool[i] for i in indices)
    while True:
        for i in reversed(range(r)):
            if indices[i] != n - 1:
                break
        else:
            return
        indices[i:] = [indices[i] + 1] * (r - i)
        yield tuple(pool[i] for i in indices)

def product_permute(*args, repeat=1):
    pools = [tuple(pool) for pool in args] * repeat
    result = [[]]
    for pool in pools:
        result = [x+[y] for x in result for y in pool]
    for prod in result:
        yield tuple(prod)

## Problem 2-1 Feature Selection



**Part a**
> Apply polynomials of order M = 1 and M = 2 over the dimension D = 8 
input data.
> Also, evaluate the corresponding RMS error on the training set and valid set.

In [4]:
train_rms_error = []
valid_rms_error = []

# Apply polynomials of order M = 1 and M = 2
for m in range(1, 3):
    train_phi_x = basis_function(train_x, m, 8)
    w = least_square(train_phi_x, train_t)
    rms_error = root_mean_square_error(train_phi_x, w, train_t)
    train_rms_error.append(rms_error)
    
    valid_phi_x = basis_function(valid_x, m, 8)
    rms_error = root_mean_square_error(valid_phi_x, w, valid_t)
    valid_rms_error.append(rms_error)

    if m == 1:
        print('Weights @ M = 1')
        print(w)
    
print('\nTraining Stage:')
print('M = 1, RMS Error =', train_rms_error[0], '\nM = 2, RMS Error =', train_rms_error[1])
print('\nValidation Stage:')
print('M = 1, RMS Error =', valid_rms_error[0], '\nM = 2, RMS Error =', valid_rms_error[1])

Weights @ M = 1
[-3.56141127e+06 -4.24450562e+04 -4.22801376e+04  1.19697293e+03
 -8.63932391e+00  1.17358660e+02 -3.82454441e+01  4.63726750e+01
  4.04914293e+04]

Training Stage:
M = 1, RMS Error = 69420.81601487994 
M = 2, RMS Error = 66666.41441382407

Validation Stage:
M = 1, RMS Error = 70793.96207221576 
M = 2, RMS Error = 68980.40899604809


**Part b**
> Analysis the weights of polynomial model M = 1 and select the most
contributive feature.

In polynomial model M = 1, there are 8 weights totally. Besides, the last weight has the largest value.

There are two methods to find the most contributive feature.

1.   Remove one of the features and compute RMS errors. Find the largest error, and the excluded feature is most contributive one. That is, without the feature, the prediction will be unreliable. 

2.   Use one of the features and compute RMS errors. Find the smallest error, and the feature is most contributive one. This method is feasible as the feature is more dominant than any other one.

I will use the second method firstly.


In [5]:
input_features = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 
                  'total_bedrooms', 'population', 'households', 'median_income']

for i in range(8):
    train_phi_x = basis_function(train_x[:, [i]], order=1, features=1)
    w = least_square(train_phi_x, train_t)
    rms_error = root_mean_square_error(train_phi_x, w, train_t)

    print("With Feature: %-18s, RMS Error = %f" %(input_features[i], rms_error))

With Feature: longitude         , RMS Error = 115678.517473
With Feature: latitude          , RMS Error = 114606.037072
With Feature: housing_median_age, RMS Error = 115111.297709
With Feature: total_rooms       , RMS Error = 114777.389042
With Feature: total_bedrooms    , RMS Error = 115672.773325
With Feature: population        , RMS Error = 115757.934509
With Feature: households        , RMS Error = 115574.487669
With Feature: median_income     , RMS Error = 83681.436014


The last feature corresponds to the smallest RMS error. Apparently, the most contributive feature is `median_income`.




## Problem 2-2 Maximum Likelihood Approach

**Part a**
> Choose one of the following basis functions, polynomial, Gaussian, Sigmoid, or hybrid, to further improve your regression model.

I choose modified `polynomial` basis function for the following reasons.


1.   Polynomial basis function has fexibility to choose the degree of order. Hence, the number of parameters, i.e., the shape of the design matrix, can be adjusted easily.
2.   Some hyperparameters in Gaussian or sigmoid basis function are not easily determined in this case.



**Part b**
> Analyze the result of the basis function introduced to linear regression model.

In [6]:
train_rms_error = []
valid_rms_error = []
order_max = 4

for m in range(1, order_max + 1):
    train_phi_x = basis_function(train_x, m, 8)
    # print(np.shape(train_x))
    # print(np.shape(train_phi_x))
    # print('')
    w = least_square(train_phi_x, train_t)
    rms_error = root_mean_square_error(train_phi_x, w, train_t)
    train_rms_error.append(rms_error)
    
    valid_phi_x = basis_function(valid_x, m, 8)
    rms_error = root_mean_square_error(valid_phi_x, w, valid_t)
    valid_rms_error.append(rms_error)

print('Training Stage:')
for i in range(order_max):
    print("M = %d, RMS Error = %f" % (i + 1, train_rms_error[i]))
print('\nValidation Stage:')
for i in range(order_max):
    print("M = %d, RMS Error = %f" % (i + 1, valid_rms_error[i]))

Training Stage:
M = 1, RMS Error = 69420.816015
M = 2, RMS Error = 66666.414414
M = 3, RMS Error = 72415.218741
M = 4, RMS Error = 99211.054617

Validation Stage:
M = 1, RMS Error = 70793.962072
M = 2, RMS Error = 68980.408996
M = 3, RMS Error = 78657.948600
M = 4, RMS Error = 149609.811942


**Part c**
> Apply N-fold cross-validation in the training stage to select at least one hyperparameter (order, parameter number, . . .) for model and do some discussion (underfitting, overfitting)

In [11]:
train_rms_error = []
valid_rms_error = []
N = 100
order_max = 3

for m in range(1, order_max + 1):
    for n in range(100):
        train_x = np.concatenate((data[:int(n * input_size / N), 0:8], data[int((n + 1) * input_size / N):, 0:8]))
        train_t = np.concatenate((data[:int(n * input_size / N), 8], data[int((n + 1) * input_size / N):, 8]))

        valid_x = data[int(n * input_size / N):int((n + 1) * input_size / N), 0:8]
        valid_t = data[int(n * input_size / N):int((n + 1) * input_size / N), 8]

        train_phi_x = basis_function(train_x, order=m, features=8)
        w = least_square(train_phi_x, train_t)

        if n == 0:
            w_av = w
        else:
            w_av += w

    w_av /= 10.0
    train_rms_error.append(root_mean_square_error(train_phi_x, w_av, train_t))
    valid_phi_x = basis_function(valid_x, order=m, features=8)
    valid_rms_error.append(root_mean_square_error(valid_phi_x, w_av, valid_t))

print('Training Stage:')
for i in range(order_max):
    print("M = %d, RMS Error = %f" % (i + 1, train_rms_error[i]))
print('\nValidation Stage:')
for i in range(order_max):
    print("M = %d, RMS Error = %f" % (i + 1, valid_rms_error[i]))

Training Stage:
M = 1, RMS Error = 2039778.002821
M = 2, RMS Error = 2047005.787079
M = 3, RMS Error = 2031744.999810

Validation Stage:
M = 1, RMS Error = 1985267.391228
M = 2, RMS Error = 1979472.020457
M = 3, RMS Error = 1968568.074199


##Problem 2-3 Maximum a posteriori approach

**Part b**
> Use maximum a posteriori approach method to retest the model designed in Problem 2-2.

In [10]:
train_rms_error = []
valid_rms_error = []
N = 10
order_max = 3

for m in range(1, order_max + 1):
    for n in range(10):
        train_x = np.concatenate((data[:int(n * input_size / N), 0:8], data[int((n + 1) * input_size / N):, 0:8]))
        train_t = np.concatenate((data[:int(n * input_size / N), 8], data[int((n + 1) * input_size / N):, 8]))

        valid_x = data[int(n * input_size / N):int((n + 1) * input_size / N), 0:8]
        valid_t = data[int(n * input_size / N):int((n + 1) * input_size / N), 8]

        train_phi_x = basis_function(train_x, order=m, features=8)
        w = regularize_least_square(train_phi_x, train_t, lambda_rls=10)

        if n == 0:
            w_av = w
        else:
            w_av += w

    w_av /= 10.0
    train_rms_error.append(root_mean_square_error(train_phi_x, w_av, train_t))
    valid_phi_x = basis_function(valid_x, order=m, features=8)
    valid_rms_error.append(root_mean_square_error(valid_phi_x, w_av, valid_t))

print('Training Stage:')
for i in range(order_max):
    print("M = %d, RMS Error = %f" % (i + 1, train_rms_error[i]))
print('\nValidation Stage:')
for i in range(order_max):
    print("M = %d, RMS Error = %f" % (i + 1, valid_rms_error[i]))

Training Stage:
M = 1, RMS Error = 73717.319546
M = 2, RMS Error = 66700.599266
M = 3, RMS Error = 75576.002544

Validation Stage:
M = 1, RMS Error = 75619.927549
M = 2, RMS Error = 68661.947325
M = 3, RMS Error = 71987.838254
